In [51]:
import pandas as pd
import requests
import pprint

import urllib.request

from pathlib import Path

import gzip
import shutil

In [52]:
# this dataset describes the metadata for each image in Landsat 9 (i think 9...)

# download this file from here:
# https://landsat.usgs.gov/landsat/metadata_service/bulk_metadata_files/LANDSAT_OT_C2_L2.csv.gz
# and extract...
# Will take a while. There is a "bulk metadata api" that describes all the images available that will replace the above file
# more info here:
# https://www.usgs.gov/landsat-missions/bulk-metadata-service


path = "./metadata/LANDSAT_OT_C2_L2.csv.gz"
csv_path = "./metadata/metadata.csv"
with gzip.open(path, 'rb') as f_in:
    with open(csv_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
data = pd.read_csv(csv_path)



In [53]:
# some conditions defining "California"
# we want to get all images that capture california
ca_min_lat = 32.5
ca_max_lat = 42
ca_min_lon = -124.5
ca_max_lon = -114.0

# define condtitions

# corner upper left
CUL_LAT = (data["Corner Upper Left Latitude"] >= ca_min_lat) & (data["Corner Upper Left Latitude"] <= ca_max_lat)
CUL_LON = (data["Corner Upper Left Longitude"] >= ca_min_lon) & (data["Corner Upper Left Longitude"] <= ca_max_lon)

# corner upper right
CUR_LAT = (data["Corner Upper Right Latitude"] >= ca_min_lat) & (data["Corner Upper Right Latitude"] <= ca_max_lat)
CUR_LON = (data["Corner Upper Right Longitude"] >= ca_min_lon) & (data["Corner Upper Right Longitude"] <= ca_max_lon)

# corner lower left
CLL_LAT = (data["Corner Lower Left Latitude"] >= ca_min_lat) & (data["Corner Lower Left Latitude"] <= ca_max_lat)
CLL_LON = (data["Corner Lower Left Longitude"] >= ca_min_lon) & (data["Corner Lower Left Longitude"] <= ca_max_lon)

# corner lower right
CLR_LAT = (data["Corner Lower Right Latitude"] >= ca_min_lat) & (data["Corner Lower Right Latitude"] <= ca_max_lat)
CLR_LON = (data["Corner Lower Right Longitude"] >= ca_min_lon) & (data["Corner Lower Right Longitude"] <= ca_max_lon)


# ca_data = data[(CUL_LAT & CUL_LON) | (CUR_LAT & CUR_LON ) | (CLL_LAT & CLL_LON) | (CLR_LAT & CLR_LON)]
ca_data = data[(data["Scene Center Latitude"] >= ca_min_lat) & (data["Scene Center Latitude"] <= ca_max_lat) & 
               (data["Scene Center Longitude"] >= ca_min_lon) & (data["Scene Center Longitude"] <= ca_max_lon)]




In [54]:
# some conditions defining "Monterey Sample Area"
# we want to get all images that capture monterey
monterey_min_lat = 35.787773
monterey_max_lat = 36.914796
monterey_min_lon = -121.973412
monterey_max_lon = -120.211479

# define condtitions

# corner upper left
CUL_LAT = (data["Corner Upper Left Latitude"] >= monterey_min_lat) & (data["Corner Upper Left Latitude"] <= monterey_max_lat)
CUL_LON = (data["Corner Upper Left Longitude"] >= monterey_min_lon) & (data["Corner Upper Left Longitude"] <= monterey_max_lon)

# corner upper right
CUR_LAT = (data["Corner Upper Right Latitude"] >= monterey_min_lat) & (data["Corner Upper Right Latitude"] <= monterey_max_lat)
CUR_LON = (data["Corner Upper Right Longitude"] >= monterey_min_lon) & (data["Corner Upper Right Longitude"] <= monterey_max_lon)

# corner lower left
CLL_LAT = (data["Corner Lower Left Latitude"] >= monterey_min_lat) & (data["Corner Lower Left Latitude"] <= monterey_max_lat)
CLL_LON = (data["Corner Lower Left Longitude"] >= monterey_min_lon) & (data["Corner Lower Left Longitude"] <= monterey_max_lon)

# corner lower right
CLR_LAT = (data["Corner Lower Right Latitude"] >= monterey_min_lat) & (data["Corner Lower Right Latitude"] <= monterey_max_lat)
CLR_LON = (data["Corner Lower Right Longitude"] >= monterey_min_lon) & (data["Corner Lower Right Longitude"] <= monterey_max_lon)


# monterey_data = data[(CUL_LAT & CUL_LON) | (CUR_LAT & CUR_LON ) | (CLL_LAT & CLL_LON) | (CLR_LAT & CLR_LON)]
monterey_data = data[(data["Scene Center Latitude"] >= monterey_min_lat) & (data["Scene Center Latitude"] <= monterey_max_lat) & 
               (data["Scene Center Longitude"] >= monterey_min_lon) & (data["Scene Center Longitude"] <= monterey_max_lon)]


# specific_lat = 35.853
# specific_lon = -119.343

# specific_lat = 36.297
# specific_lon = -119.648

# specific_lat = 35.875503
# specific_lon = -119.576063

# monterey_data = data[
#     (data["Corner Upper Right Latitude"] >= specific_lat | data["Corner Upper Left Latitude"] >= specific_lat) & 
#     (specific_lat >= data["Corner Lower Right Latitude"]) &
#     (data["Corner Upper Right Longitude"] >= specific_lon) & ( specific_lon >= data["Corner Lower Right Longitude"])
#     ]




In [55]:
monterey_display_ids=ca_data[
    (ca_data["Scene Center Latitude"] < 37) & (ca_data["Scene Center Latitude"] > 36) &
    (ca_data["Scene Center Longitude"] > -122.1) & (ca_data["Scene Center Longitude"] < -121.0)
    ]["Display ID"].to_list()

# ca_display_ids=ca_data["Display ID"].to_list()

In [56]:
len(monterey_display_ids)

104

In [57]:
API_HOST = "https://espa.cr.usgs.gov/api/v1"
USERNAME="joshuafram@berkeley.edu"
PASSWORD="Agrisense1!!"

## Datasets

### NASA EarthData - [Agriculture](https://www.earthdata.nasa.gov/learn/pathfinders/agricultural-and-water-resources-data-pathfinder)

### Landsat Collection 2 - Surface Reflectance-derived Spectral Indices - [Overview](https://www.usgs.gov/landsat-missions/landsat-surface-reflectance-derived-spectral-indices)

Use the [M2M Api](https://m2m.cr.usgs.gov/) or the Bulk Downloader


- Normalized Difference Vegetation Index - (NDVI)
    - NDVI measures "healthy vegetation" by taking the difference in between near-infrared (reflected) and red (absored) light. 
- Enhanced Soil Vegetation Index (EVI)
    - Similar to NDVI, but also corrects for *atmospheric conditions* and *canopy background noise*.
    - More sensitive to dense vegetation areas
- Soil Adjusted Vegetation Index (SAVI)
    - Applies corrections to NDVI for the influence of *soil brightness* in areas where vegetation cover is low.
- Modified Soil Adjusted Vegetation Index (MSAVI)
    - Minimizes the effect of *bare soil* from the SAVI
- Normalized Difference Moisture Index (NDMI)
    - determines vegetation water content
- Dynamic Surface Water Extent (don't think we need this actually...)
    - provides information about the present and condition of surface water.
- Surface Temperature
    - temperature of surface is important indicator of crop and vegetation health.

### EROS satellite
- eVIIRS GLOBAL

In [58]:
def request_data(url:str, json:bool=True, body:dict|None=None, method:str="get"):  
    print("Sending request:")
    print(url)
    if method == "get":
        resp = requests.get(url, auth=(USERNAME, PASSWORD), json=body)
    elif method == "post":
        resp = requests.post(url, auth=(USERNAME, PASSWORD), json=body)
    elif method == "put":
        resp = requests.put(url, auth=(USERNAME, PASSWORD), json=body)
    else:
        return "METHOD NOT SUPPORTED"

    if json:
        return resp.json()
    else:
        return resp

    


In [59]:
endpoint = "/"
response = requests.get(API_HOST + endpoint)
print(response.text)


{"messages":{"errors":["Invalid username/password"]}}



In [60]:
endpoint = "/"
data = request_data(API_HOST + endpoint)
pprint.pprint(data)

Sending request:
https://espa.cr.usgs.gov/api/v1/
{'description': 'Version 1 of the ESPA API',
 'operations': {'/api': {'function': 'list versions',
                         'methods': ['HEAD', 'GET']},
                '/api/v1': {'function': 'list operations',
                            'methods': ['HEAD', 'GET']},
                '/api/v1/available-products': {'comments': 'sceneids should be '
                                                           'delivered in the '
                                                           'product_ids '
                                                           'parameter, comma '
                                                           'separated if more '
                                                           'than one',
                                               'function': 'list available '
                                                           'products per '
                                                           'scene

In [61]:

#default example
# display_id = "LC08_L2SP_042033_20200912_20200919_02_T1"

#monterey example
display_id = "LC09_L2SP_043035_20211121_20230407_02_T1"
print(ca_data[ca_data['Display ID'] == display_id])
print(ca_data[ca_data['Display ID'] == display_id]['Date Acquired'].max())

                                               Browse Link  \
2377236  https://landsatlook.usgs.gov/gen-browse?size=r...   

                                       Display ID            Ordering ID  \
2377236  LC09_L2SP_043035_20211121_20230407_02_T1  LC90430352021325LGN04   

                    Landsat Product Identifier L2  \
2377236  LC09_L2SP_043035_20211121_20230407_02_T1   

                    Landsat Product Identifier L1 Landsat Scene Identifier  \
2377236  LC09_L1TP_043035_20211121_20230407_02_T1    LC90430352021325LGN04   

        Date Acquired Collection Category  Collection Number  WRS Path  ...  \
2377236    2021/11/21                  T1                  2        43  ...   

         Scene Center Latitude  Scene Center Longitude  \
2377236               36.04317              -121.64178   

         Corner Upper Left Latitude Corner Upper Left Longitude  \
2377236                    37.09576                   -122.8976   

         Corner Upper Right Latitude Corner Upp

In [62]:
endpoint = "/available-products"
# avail_list = {
#     "inputs": [
#         "LE07_L2SP_038034_20050424_20200914_02_T2"
#     ]
# }
avail_list = {
    "inputs": [display_id] 
}
data = request_data(API_HOST + endpoint, body=avail_list)
pprint.pprint(data)

Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
{'olitirs9_collection_2_l2': {'inputs': ['LC09_L2SP_043035_20211121_20230407_02_T1'],
                              'products': ['l1',
                                           'sr_ndvi',
                                           'sr_evi',
                                           'sr_savi',
                                           'sr_msavi',
                                           'sr_ndmi',
                                           'sr_nbr',
                                           'sr_nbr2',
                                           'sr_ndsi',
                                           'et']}}


In [63]:
def update_nested_key(data, target_key, new_value):
    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                data[key] = new_value
            elif isinstance(value, dict):
                update_nested_key(value, target_key, new_value)
            elif isinstance(value, list):
                for item in value:
                    update_nested_key(item, target_key, new_value)
    elif isinstance(data, list):
        for item in data:
            update_nested_key(item, target_key, new_value)

In [64]:
# now do the above for all landsat images (UPDATED TO PULL more products)

for display_id in monterey_display_ids:
    
    endpoint = "/available-products"
    avail_list = {
        "inputs": [display_id] 
    }
    data = request_data(API_HOST + endpoint, body=avail_list)

    endpoint = "/order"
    data['format'] = 'gtiff'
    update_nested_key(data, 'products', ['l1','sr_ndmi','et'])
    resp = request_data(API_HOST + endpoint, body=data, method="post")
    # pprint.pprint(resp)

Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:
https://espa.cr.usgs.gov/api/v1/available-products
Sending request:
https://espa.cr.usgs.gov/api/v1/order
Sending request:

In [65]:
# get all orders

endpoint = "/list-orders"

all_orders = request_data(API_HOST + endpoint)
# pprint.pprint(all_orders)


Sending request:
https://espa.cr.usgs.gov/api/v1/list-orders


In [66]:
#add order filter code block
def filter_list_by_string(input_list, filter_string):
    return [item for item in input_list if filter_string in item]

In [67]:

# filter for only orders made on target day
#CHANGE string to current date if re-running script!
all_orders_filtered = filter_list_by_string(all_orders, '07042024')

In [68]:
# get download link for all finished orders
# GIVE TIME FOR ORDERS TO BE FINISHED

order_dict = {}
i=0

for orderid in all_orders_filtered:

    # get status first
    endpoint = f"/item-status/{orderid}"
    resp = request_data(API_HOST + endpoint)
    # pprint.pprint(resp)
    print(i)
    i+=1
    for product in resp[orderid]:

        if product["status"] == "complete":
            download_url = product["product_dload_url"]
            name = product["name"]
            order_dict[orderid] = {'name':name,'download_url':download_url,'year':ca_data[ca_data['Display ID'] == name]['Date Acquired'].max()}
        

    

Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164246-327
0
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164245-090
1
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164243-901
2
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164242-146
3
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164240-917
4
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164239-621
5
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164238-413
6
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164237-159
7
Sending request:
https://espa.cr.usgs.gov/api/v1/item-status/espa-joshua.fram@gmail.com-07042024-164235-987
8
Sending re

In [71]:
for order_id, order_details in order_dict.items():
    if 'year' in order_details and isinstance(order_details['year'], str):
        order_details['year'] = order_details['year'].replace('/', '.')

In [ ]:
#download orders by download url

# county = 'Monterey'
# for order in order_dict:
#     name=order_dict[order]['name']
#     download_url=order_dict[order]['download_url']
#     date=order_dict[order]['year']
#     print(name)
#     print(download_url)
#     print(date)

#     data_dir = Path("./data/landsat_monterey")
#     data_dir.mkdir(exist_ok=True)
#     urllib.request.urlretrieve(download_url,data_dir / f"{county}_{date}_{name}.tar.gz" )

In [75]:
import urllib.request
from pathlib import Path
import os
import boto3
from botocore.exceptions import NoCredentialsError

# Initialize S3 client
s3_client = boto3.client('s3')

# S3 bucket name and folder
bucket_name = 'agrisense'
folder_name = 'landsat'

def upload_to_s3(file_path, bucket, folder, object_name=None):
    """Upload a file to an S3 bucket

    :param file_path: File to upload
    :param bucket: Bucket to upload to
    :param folder: Folder in the bucket to upload to
    :param object_name: S3 object name. If not specified, then file_path is used
    :return: True if file was uploaded, else False
    """
    if object_name is None:
        object_name = file_path.name

    s3_key = f"{folder}/{object_name}"

    try:
        s3_client.upload_file(str(file_path), bucket, s3_key)
        print(f"File {file_path} uploaded to {bucket}/{s3_key}")
        return True
    except NoCredentialsError:
        print("Credentials not available")
        return False

county = 'Monterey'

data_dir = Path("./data/landsat_monterey")
data_dir.mkdir(parents=True, exist_ok=True)

for order in order_dict:
    name = order_dict[order]['name']
    download_url = order_dict[order]['download_url']
    date = order_dict[order]['year']
    print(name)
    print(download_url)
    print(date)

    local_file_path = data_dir / f"{county}_{date}_{name}.tar.gz"
    
    # Download the file
    try:
        urllib.request.urlretrieve(download_url, local_file_path)
    except:
        continue
    
    # Upload to S3
    if upload_to_s3(local_file_path, bucket_name, folder_name, f"{county}_{date}_{name}.tar.gz"):
        # Delete the local file if upload was successful
        os.remove(local_file_path)
        print(f"Local file {local_file_path} deleted.")
    else:
        print(f"Failed to upload {local_file_path}. Local file not deleted.")


LC08_L2SP_043035_20171029_20200902_02_T1
https://edclpdsftp.cr.usgs.gov/orders/espa-joshua.fram@gmail.com-07042024-162507-839/LC080430352017102902T1-SC20240704212950.tar.gz
2017.10.29
File data/landsat_monterey/Monterey_2017.10.29_LC08_L2SP_043035_20171029_20200902_02_T1.tar.gz uploaded to agrisense/landsat/Monterey_2017.10.29_LC08_L2SP_043035_20171029_20200902_02_T1.tar.gz
Local file data/landsat_monterey/Monterey_2017.10.29_LC08_L2SP_043035_20171029_20200902_02_T1.tar.gz deleted.
LC08_L2SP_043035_20171013_20200902_02_T1
https://edclpdsftp.cr.usgs.gov/orders/espa-joshua.fram@gmail.com-07042024-162506-531/LC080430352017101302T1-SC20240704212950.tar.gz
2017.10.13
File data/landsat_monterey/Monterey_2017.10.13_LC08_L2SP_043035_20171013_20200902_02_T1.tar.gz uploaded to agrisense/landsat/Monterey_2017.10.13_LC08_L2SP_043035_20171013_20200902_02_T1.tar.gz
Local file data/landsat_monterey/Monterey_2017.10.13_LC08_L2SP_043035_20171013_20200902_02_T1.tar.gz deleted.
LC08_L2SP_043035_20170404_